<a href="https://colab.research.google.com/github/tomonari-masada/course2022-nlp/blob/main/03_elementary_Japanese_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **日本語データの扱い方**

* 日本語のテキストは空白によって単語に分たれていない。
 * 同じように、テキストが空白によって単語へ分たれていない言語は？
* そのため、まず最初にテキストを単語へ分割する必要がある。
* この作業を**形態素解析**と言う。

* 長い文字列としてのテキストを、より細かい単位へ分割することを、一般にtokenizationと言う。
* tokenizationの単位は、単語のように意味的なまとまりを持つ単位とは限らない。
 * サブワードは、意味的なまとまりを持たない（単独のサブワードを見ただけでは意味が分からないことが多い）。

## spaCyで形態素解析

* ここでは、spaCyを介してSudachiという形態素解析器を使う。
 * 形態素解析器としては、他には、MeCabやJUMANが有名。

In [ ]:
!pip install spacy[ja]

* spaCyの日本語版をインストールすると、sudachiがインストールされる。
 * 以下のように、コマンドラインからでも使えるようになっている。

In [ ]:
!echo "すもももももももものうち" | sudachipy

### spaCyの日本語統計モデルをロード

In [ ]:
from spacy.lang.ja import Japanese

nlp = Japanese()

### spaCyからSudachiを利用
* 分かち書きされた単語そのもの以外の、品詞などの情報も表示させている。
 * tagに「形容動詞」は無く、その語幹が「形状詞」とされている。
 * 「じゃない」の「ない」は、tagでは「形容詞」だが、posでは「AUX」である。

In [ ]:
doc = nlp('吾輩は猫である。名前はまだ無い。')
print(doc.text)

In [ ]:
for token in doc:
  print(f'text:{token.text}, pos:{token.pos_}, tag:{token.tag_}, lemma:{token.lemma_}')

## Mecab+NEologdで形態素解析
* MeCabは日本語の形態素解析ツール。有名。
* [NEologd](https://github.com/neologd/mecab-ipadic-neologd)はneologisms（新表現）にも対応した日本語の辞書。
 * 最近は更新されていないらしい。

### MeCabのインストール

In [ ]:
!apt-get install mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8

### NEologdのインストール

* `file`というコマンドをインストールしておく。

In [ ]:
!sudo apt install file

* 以下のセルの実行が必要（かもしれない）。

In [ ]:
!rm -rf /content/mecab-ipadic-neologd/libexec/../build/mecab-ipadic-2.7.0-20070801
!rm /content/mecab-ipadic-neologd/libexec/../build/mecab-ipadic-2.7.0-20070801.tar.gz

In [ ]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n 

* NEologdがどこにインストールされたかを調べる。

In [ ]:
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

* `mecabrc`というファイルへのパスが後で必要になるので、調べておく。

In [ ]:
!find / -iname mecabrc

### mecabコマンドによる形態素解析

In [ ]:
!echo "すもももももももものうち" | mecab

In [ ]:
!echo "人工知能科学研究科" | mecab

In [ ]:
!echo "人工知能科学研究科" | mecab -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd

### PythonからMeCabを利用

In [ ]:
!pip install mecab-python3

* mecabrcへのパスを`-r`オプションで指定してからMeCabを使う。

In [ ]:
import os
import MeCab

mecab = MeCab.Tagger('-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -r /etc/mecabrc')

In [ ]:
node = mecab.parseToNode("吾輩は猫である。名前はまだ無い。")
while node:
  print(f'{node.surface}\t{node.feature}')
  node = node.next

* 品詞だけ取り出す

In [ ]:
node = mecab.parseToNode("吾輩は猫である。名前はまだ無い。")
while node:
  features = node.feature.split(',')
  word = features[6]
  if word == '*':
    word = node.surface
  if len(word):
    print(f'{word}\t{features[0]}')
  node = node.next

# 課題３

* （この課題は、宿題ではなく、いまここで作業しつつ解いてしまいます。）
* Wikipediaの複数の記事を、lemmaを半角スペースでつないだ、長い文字列へ変換する。
 * ここでは、コンピュータ科学の様々な分野の記事を題材として使う。
* scikit-learnの`TfidfVectorizer`を使って、各記事における単語の出現頻度からなる文書ベクトルを得る。
* 特徴ベクトルどうしの類似度を計算し、「人工知能」分野と最も似ている順に　３つの分野がどの分野かを求める。
 * 答えは自分の感覚でチェック。
 * 文書ベクトルを作る時に、単語の品詞を名詞に限定するなど、品詞の情報を使うことで結果を改善できるかどうかも、余裕があれば試行錯誤する。

## 課題の手順(1)

### Wikipediaのエントリをダウンロードして形態素解析を適用

* Wikipediaの「人工知能」エントリをダウンロードする。
 * https://ja.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BD
* そして、そこに含まれる段落（__`<p>`__タグで囲まれた範囲）を列挙する。
* 各段落のテキストに形態素解析を適用する。
* 形態素解析で得られたlemmaを半角スペースでつないで、エントリ全体をひとつの長い文字列にする。
 * text、つまり、単語に分たれたそのままの文字列を半角スペースでつなぐのではない。
 * lemma（原型に戻したもの）を半角スペースでつなぐこと。
 * posやtagを見て、不要そうな単語を適当に削除してもよい。


* 「人工知能」エントリをダウンロードしてparserのインスタンスを作る。

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

url = 'https://ja.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BD'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

* 段落のテキストを取得する。

In [ ]:
lines = list()
for para in soup.find_all('p'):
  lines.append(para.text)

In [ ]:
lines

* Sudachiで形態素解析し、分かち書き後のlemmaを取得する。

In [ ]:
x_pos = ['SPACE', 'PUNCT', 'AUX', 'ADP', 'SYM', 'DET', 'SCONJ', 'PART'] # 除去する品詞
tokens = list()
for line in lines[:10]:
  for token in nlp(line):
    pos = token.pos_
    if not pos in x_pos:
      print(f'text:{token.text}, pos:{token.pos_}, tag:{token.tag_}, lemma:{token.lemma_}')
      tokens.append(token.lemma_)

* すべてのlemmaを半角スペースでつないで、長い文字列にする。

In [ ]:
doc_AI = ' '.join(tokens)
print(doc_AI)

* 上記の操作をまとめておこなう関数を定義しておく。
 * 後で、各エントリについて、この関数を呼び出す。

In [ ]:
def morph(soup, nlp):

  lines = list()
  for para in soup.find_all('p'):
    lines.append(para.text)

  x_pos = ['SPACE', 'PUNCT', 'AUX', 'ADP', 'SYM', 'DET', 'SCONJ', 'PART']
  tokens = list()
  for line in lines:
    for token in nlp(line):
      pos = token.pos_
      if pos not in x_pos:
        tokens.append(token.lemma_)

  return ' '.join(tokens)

## 課題の手順(2)

### 「コンピュータ科学」の様々な分野に関するWikipediaエントリについて上記の作業を実行
*  「人工知能」エントリの下部にある「コンピュータ科学」の分野一覧から、aタグのhref属性にあるURLを抜き出す。
* ただし、__`/wiki/`__という文字列で始まっているURLであり、かつ、テンプレートの状態でないものだけを抜き出す。
 * Wikipediaのクローリングについてもっと詳しく知りたい場合は、下記のページ等を参照されたい。
   * https://medium.com/@robinlphood/tutorial-a-simple-crawler-for-wikipedia-d7b6f6f55d5

* 「人工知能」エントリの下部にある「コンピュータ科学」の分野一覧から、aタグのhref属性にあるURLを抜き出す。


In [ ]:
target_str = '表話編歴コンピュータ科学'
prefix = '/wiki/'

urls = dict()
for table in soup.find_all('table'):
  if table.text[:len(target_str)] != target_str: continue
  for td in table.find_all('td'):
    for a in td.find_all('a'):
      if not a.text: continue
      try:
        if a.text.find('英語版') == -1:
          href = a['href']
          if href[:len(prefix)] == prefix and href.find('/Template:') == -1 and href.find('/Category:') == -1:
            urls[a.text] = 'https://ja.wikipedia.org' + href
      except:
        continue

for k in urls:
  print(k, urls[k])

* 各エントリをダウンロードしてparserのインスタンスを作り、辞書として保存。



In [ ]:
from tqdm import tqdm

soups = dict()
for k in tqdm(urls):
  html = urlopen(urls[k]) 
  soups[k] = BeautifulSoup(html, 'html.parser')

* 先ほど定義した関数morph()を使って各エントリを形態素解析し、lemmaが半角スペースで区切られた文字列へ変換する。


In [ ]:
genre = list()
corpus = list()
for k in tqdm(soups):
  genre.append(k)
  doc = morph(soups[k], nlp)
  corpus.append(doc)

* 再利用するために、csvファイルとして保存しておく。


In [ ]:
import pandas as pd

genre.append('人工知能')
corpus.append(doc_AI)

df = pd.DataFrame(list(zip(genre, corpus)), columns=['genre', 'text'])
print(df.head())
df.to_csv('cs_corpus.csv')

## 課題の手順(3)

### scikit-learnのTfidfVectorizerで単語の出現頻度を要素とするベクトルを作成
* これにより、各文書のベクトル表現が得られる。
* 興味のある対象のベクトル表現を得ることは、その対象を機械学習アルゴリズムの入力データとして使うための第一歩。

* 先ほど作成したcsvファイルを読んで、textカラムをTfidfVectorizerのインスタンスでベクトル化する。
 * TfidfVectorizerのmin_dfやmax_dfをチューニングすると、より面白い結果になる可能性あり。

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('cs_corpus.csv')

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text']).toarray()

In [ ]:
print('文書数:{}; 語彙サイズ：{}'.format(*X.shape))

* 語彙を取得

In [ ]:
vocab = vectorizer.get_feature_names_out()
print(vocab)

* TF-IDF値を全文書にわたって和をとった値の大きい順で上位20単語を見てみる
 * コーパス全体で重要な単語を見ていることになる。

In [ ]:
import numpy as np

vocab = np.array(vocab)
print(vocab[np.argsort(- X.sum(axis=0))][:20])

* 「人工知能」エントリと他のエントリとの距離を求める。
 * 注：scipy.spatial.distance.cosineは、cosine類似度を1から引いたもの。

In [ ]:
genre = df['genre'].values.tolist()
print(', '.join(genre))

In [ ]:
from scipy.spatial import distance

index_AI = genre.index('人工知能')
print(f'「{genre[index_AI]}」と「{genre[7]}」との間での・・・')
print(f'ユークリッド距離: {np.linalg.norm(X[0] - X[index_AI])}')
print(f'内積: {np.dot(X[0], X[index_AI])}')
print(f'コサイン距離: {distance.cosine(X[0], X[index_AI])}')

### ここから各自作業。
* 「人工知能」と、Wikipediaのエントリの類似度の上で、最も近いジャンルは、どれ？
* ユークリッド距離とコサイン距離のうち、どちらを類似度として使うのが良さそうか？